In [ ]:
! which python

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from timeseries import IterateOutputCESM
from paths import path_results
from xr_regression import xr_linear_trends_2D
from maps import map_robinson

In [ ]:
# example
file_ex = next(IterateOutputCESM('ocn_rect', 'ctrl', 'yrly', name='SHF'))[2]
ds = xr.open_dataset(file_ex, decode_times=False)
ds.SHF.plot()

In [ ]:
# stacking files into one xr Dataset object
for run in ['ctrl', 'rcp']:
    for i, (y,m,s) in enumerate(IterateOutputCESM('ocn_rect', run, 'yrly', name='SHF')):
        ds = xr.open_dataset(s, decode_times=False)
        if i==0:
            ds_new = ds.copy()
        else:
            ds_new = xr.concat([ds_new, ds], dim='time')
    ds_new.to_netcdf(f'{path_results}/SHF/SHF_yrly_{run}.nc')

In [ ]:
SHF_ctrl = xr.open_dataset(f'{path_results}/SHF/SHF_yrly_ctrl.nc', decode_times=False)
SHF_rcp  = xr.open_dataset(f'{path_results}/SHF/SHF_yrly_rcp.nc' , decode_times=False)

In [ ]:
SHF_ctrl

In [ ]:
SHF_ctrl.SHF[0,0,0]<0

# Global time series

In [ ]:
from xr_DataArrays import generate_xr_AREA

In [ ]:
AREA = generate_xr_AREA('ocn_rect')

In [ ]:
AREA.plot()

In [ ]:
# surface integral
sec_per_year = 3600*24*365

SHF_imbal_ctrl = sec_per_year*(AREA.where(SHF_ctrl.SHF<500)*SHF_ctrl.SHF).sum(dim=['lat', 'lon'])  # [J/year]
SHF_imbal_rcp  = sec_per_year*(AREA.where(SHF_rcp.SHF <500)*SHF_rcp.SHF ).sum(dim=['lat', 'lon'])

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot(SHF_imbal_ctrl/1e21, lw=2, label='CTRL')
plt.plot(SHF_imbal_rcp /1e21, lw=2, label='RCP')
plt.legend(ncol=3, frameon=False, fontsize=16)
plt.ylabel('SHF imbalance [ZJ/year]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/SHF/SHF_integrated_imbalance')

In [ ]:
plt.plot(SHF_ctrl.lat[1:]-SHF_ctrl.lat[:-1])
len()
plt.figure()
plt.plot(AREA[:,0])

# trends

In [ ]:
SHF_trend_ctrl = xr_linear_trends_2D(SHF_ctrl.SHF[:,:10,:10], ('lat', 'lon'))
# produces LinAlg error

In [ ]:
SHF_ctrl

In [ ]:
label = 'Surface heat flux [W/m$^2$]'
minv, maxv = -250, 250
cmap = 'RdBu_r'
filename = f'{path_results}/SHF/SHF_ctrl_mean'

f = map_robinson(xr_DataArray=SHF_ctrl.SHF[:,:,:].mean(dim='time'),
             cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)

In [ ]:
SHF_ctrl_diff = SHF_ctrl.SHF[-30:,:,:].mean(dim='time')-SHF_ctrl.SHF[:30,:,:].mean(dim='time')

label = 'Surface heat flux [W/m$^2$]'
minv, maxv = -25, 25
cmap = 'RdBu_r'
filename = f'{path_results}/SHF/SHF_ctrl_last_minus_first_30yrs'

f = map_robinson(xr_DataArray=SHF_ctrl_diff,
             cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)

In [ ]:
SHF_rcp_ctrl = SHF_rcp.SHF[-10:,:,:].mean(dim='time')-SHF_ctrl.SHF[:,:,:].mean(dim='time')

label = 'Surface heat flux [W/m$^2$]'
minv, maxv = -50, 50
cmap = 'RdBu_r'
filename = f'{path_results}/SHF/SHF_rcp_last_10_minus_ctrl_avg'

f = map_robinson(xr_DataArray=SHF_rcp_ctrl,
             cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)

# Why is the SHF negative

In [ ]:
SHF_rcp.SHF[0,:,:].sum()

In [ ]:
SHF_rcp.SHF[0,:,:].plot()

In [ ]:
from paths import file_ex_ocn_ctrl, file_ex_ocn_rect

In [ ]:
ds_hr = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
ds_lr = xr.open_dataset(file_ex_ocn_rect, decode_times=False)

In [ ]:
ds_hr

In [ ]:
(ds_hr.SHF*ds_hr.TAREA).sum(dim=('nlat','nlon'))

In [ ]:
ds_hr.SHF.plot()

In [ ]:
ds_lr.SHF.plot()

In [ ]:
ds_hr.PD